<a href="https://colab.research.google.com/github/NAshwinKumar/credit_risk_assessment/blob/master/HGANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler, LabelEncoder 
from collections import defaultdict

In [0]:
data = pd.read_excel('https://github.com/NAshwinKumar/credit_risk_assessment/raw/master/GermanCredit.xls', sheet_name='Data')

# Exploratory Data Analysis and Pre-processing

In [30]:
print ('\033[1m' + "Number of Rows     : " +'\033[0m', data.shape[0])
print ('\033[1m' + "\nNumber of Columns  : " +'\033[0m', data.shape[1])
print ('\033[1m' + "\nMissing values     :  "+'\033[0m', data.isnull().values.any())
print('\033[1m' + "\nColumn Names\t\t\tUnique Values  \n"+'\033[0m', data.nunique(),sep="")
print('\033[1m' + "\nColumn Names\t\t\tData Types  \n"+'\033[0m', data.dtypes,sep="")

Number of Rows     :  1000

Number of Columns  :  31

Missing values     :   False

Column Names			Unique Values  
CHK_ACCT              4
DURATION             33
HISTORY               5
NEW_CAR               2
USED_CAR              2
FURNITURE             2
RADIO/TV              2
EDUCATION             2
RETRAINING            2
AMOUNT              921
SAV_ACCT              5
EMPLOYMENT            5
INSTALL_RATE          4
MALE_DIV              2
MALE_SINGLE           2
MALE_MAR_or_WID       2
CO-APPLICANT          2
GUARANTOR             2
PRESENT_RESIDENT      4
REAL_ESTATE           2
PROP_UNKN_NONE        2
AGE                  53
OTHER_INSTALL         2
RENT                  2
OWN_RES               2
NUM_CREDITS           4
JOB                   4
NUM_DEPENDENTS        2
TELEPHONE             2
FOREIGN               2
RESPONSE              2
dtype: int64

Column Names			Data Types  
CHK_ACCT            float64
DURATION            float64
HISTORY             float64
NEW_CAR        

In [0]:
data = data.drop(['OBS#'], axis=1)

## Min-max normalisation

In [0]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_values = scaler.fit_transform(data.iloc[:,:-1]) 
data.iloc[:,:-1] = scaled_values

In [31]:
data

,CHK_ACCT,DURATION,HISTORY,NEW_CAR,USED_CAR,FURNITURE,RADIO/TV,EDUCATION,RETRAINING,AMOUNT,SAV_ACCT,EMPLOYMENT,INSTALL_RATE,MALE_DIV,MALE_SINGLE,MALE_MAR_or_WID,CO-APPLICANT,GUARANTOR,PRESENT_RESIDENT,REAL_ESTATE,PROP_UNKN_NONE,AGE,OTHER_INSTALL,RENT,OWN_RES,NUM_CREDITS,JOB,NUM_DEPENDENTS,TELEPHONE,FOREIGN,RESPONSE
0,0.000000,0.029412,1.00,0.0,0.0,0.0,1.0,0.0,0.0,0.050567,1.00,1.00,1.000000,0.0,1.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.857143,0.0,0.0,1.0,0.333333,0.666667,0.0,1.0,0.0,1
1,0.333333,0.647059,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.313690,0.00,0.50,0.333333,0.0,0.0,0.0,0.0,0.0,0.333333,1.0,0.0,0.053571,0.0,0.0,1.0,0.000000,0.666667,0.0,0.0,0.0,0
2,1.000000,0.117647,1.00,0.0,0.0,0.0,0.0,1.0,0.0,0.101574,0.00,0.75,0.333333,0.0,1.0,0.0,0.0,0.0,0.666667,1.0,0.0,0.535714,0.0,0.0,1.0,0.000000,0.333333,1.0,0.0,0.0,1
3,0.000000,0.558824,0.50,0.0,0.0,1.0,0.0,0.0,0.0,0.419941,0.00,0.75,0.333333,0.0,1.0,0.0,0.0,1.0,1.000000,0.0,0.0,0.464286,0.0,0.0,0.0,0.000000,0.666667,1.0,0.0,0.0,1
4,0.000000,0.294118,0.75,1.0,0.0,0.0,0.0,0.0,0.0,0.254209,0.00,0.50,0.666667,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,1.0,0.607143,0.0,0.0,0.0,0.333333,0.666667,1.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1.000000,0.117647,0.50,0.0,0.0,1.0,0.0,0.0,0.0,0.081765,0.00,0.75,0.666667,0.0,0.0,0.0,0.0,0.0,1.000000,1.0,0.0,0.214286,0.0,0.0,1.0,0.000000,0.333333,0.0,0.0,0.0,1
996,0.000000,0.382353,0.50,0.0,1.0,0.0,0.0,0.0,0.0,0.198470,0.00,0.50,1.000000,1.0,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.375000,0.0,0.0,1.0,0.000000,1.000000,0.0,1.0,0.0,1
997,1.000000,0.117647,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.030483,0.00,1.00,1.000000,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.339286,0.0,0.0,1.0,0.000000,0.666667,0.0,0.0,0.0,1
998,0.000000,0.602941,0.50,0.0,0.0,0.0,1.0,0.0,0.0,0.087763,0.00,0.50,1.000000,0.0,1.0,0.0,0.0,0.0,1.000000,0.0,1.0,0.071429,0.0,0.0,0.0,0.000000,0.666667,0.0,1.0,0.0,0


In [0]:
numeric_attributes = ['DURATION','AMOUNT', 'INSTALL_RATE', 'AGE', 'NUM_CREDITS', 'NUM_DEPENDENTS']

#Feature Selection

In [0]:
def compute_gini_feature(feature):#for computing gini of an attribute
    probs = feature.value_counts(normalize=True)
    return 1 - np.sum(np.square(probs))

def compute_gini(X, y, numeric_attributes):
    # target_gini = compute_gini_feature(y)
    # print(target_gini)
    gini_index = {} #gini index of all features will be stored in this dictionary
    
    
    for  feature in X.columns:
        feature_levels = np.sort(X[feature].unique())
        # print(feature)
        for i in range(0,len(feature_levels)):
            # print(feature_levels[i])
            if feature not in numeric_attributes:
                data_split_1 = X[feature] == feature_levels[i]
                data_split_2 = X[feature] != feature_levels[i]

            else:
                if i==0:
                    continue
                data_split_1 = X[feature] < ((feature_levels[i-1]+feature_levels[i])/2)
                data_split_2 = X[feature] >= ((feature_levels[i-1]+feature_levels[i])/2)

            gini_split_1 = compute_gini_feature(y[data_split_1])
            gini_split_2 = compute_gini_feature(y[data_split_2])

            gini_feature_split = len(y[data_split_1])/len(y)*gini_split_1 + len(y[data_split_2])/len(y)*gini_split_2#gini index of a particular feature for a particular split
            gini_index[feature] = min(gini_feature_split, gini_index.get(feature,1))
            # print(gini_split_1, gini_split_2)
            # print(gini_feature_split)

    return gini_index

In [0]:
def compute_info_gain_feature(feature):#for computing info_gain of an attribute
    probs = feature.value_counts(normalize=True)
    return -1 * np.sum(np.log2(probs) * probs)

def compute_info_gain(X, y):
    info_gain = {} #info_gain index of all features will be stored in this dictionary
    target_entropy = compute_info_gain_feature(y)

    for  feature in X.columns:
        entropy_list = list()
        weight_list = list()

        feature_levels = np.sort(X[feature].unique())
        for i in range(0,len(feature_levels)):
            indices_feature_level = X[feature] == feature_levels[i]
            entropy_level = compute_info_gain_feature(y[indices_feature_level])
            entropy_list.append(round(entropy_level, 4))
            weight_level = len(y[indices_feature_level]) / len(y)
            weight_list.append(round(weight_level, 4))

        info_gain[feature] = target_entropy - np.sum(np.array(entropy_list) * np.array(weight_list))

    return info_gain

In [0]:
def compute_gain_ratio(X, y, info_gain):
    gain_ratio = {} #info_gain index of all features will be stored in this dictionary

    for  feature in X.columns:
        
        split_info_list = list()

        feature_levels = np.sort(X[feature].unique())
        for i in range(0,len(feature_levels)):
            indices_feature_level = X[feature] == feature_levels[i]

            split_info_level = -1 * ( len(y[indices_feature_level]) / len(y) ) * math.log2( len(y[indices_feature_level]) / len(y) )
            split_info_list.append(round(split_info_level, 4))

        gain_ratio[feature] = info_gain[feature] / np.sum(np.array(split_info_list))

    return gain_ratio

In [99]:
gini_index = compute_gini(data.drop('RESPONSE',axis=1),data['RESPONSE'], numeric_attributes)
gini_index_sorted = sorted(gini_index.items(), key=lambda x: x[1])#[:2]
selected_features_gini = [i[0] for i in gini_index_sorted[:12]]
print("Features selected using gini index:", selected_features_gini)

selected_features_indices_gini = [data.columns.get_loc(c) for c in selected_features_gini if c in data]
selected_features_indices_gini.sort()
selected_features_indices_gini = [i+2 for i in selected_features_indices_gini]
print("Indices of features selected using gini index:", selected_features_indices_gini)

features selected using gini index: ['CHK_ACCT', 'HISTORY', 'DURATION', 'AMOUNT', 'SAV_ACCT', 'OWN_RES', 'AGE', 'PROP_UNKN_NONE', 'REAL_ESTATE', 'OTHER_INSTALL', 'RADIO/TV', 'EMPLOYMENT']
Indices of features selected using gini index: [2, 3, 4, 8, 11, 12, 13, 21, 22, 23, 24, 26]


In [111]:
info_gain = compute_info_gain(data.drop('RESPONSE',axis=1),data['RESPONSE'])
info_gain_sorted = sorted(info_gain.items(), key=lambda x: x[1], reverse=True)
selected_features_info_gain = [i[0] for i in info_gain_sorted[:12]]
print("Features selected using info_gain:", selected_features_info_gain)

selected_features_indices_info_gain = [data.columns.get_loc(c) for c in selected_features_info_gain if c in data]
selected_features_indices_info_gain.sort()
selected_features_indices_info_gain = [i+2 for i in selected_features_indices_info_gain]
print("Indices of features selected using info_gain:", selected_features_indices_info_gain)

Features selected using info_gain: ['AMOUNT', 'CHK_ACCT', 'DURATION', 'AGE', 'HISTORY', 'SAV_ACCT', 'EMPLOYMENT', 'OWN_RES', 'PROP_UNKN_NONE', 'REAL_ESTATE', 'OTHER_INSTALL', 'RADIO/TV']
Indices of features selected using info_gain: [2, 3, 4, 8, 11, 12, 13, 21, 22, 23, 24, 26]


In [121]:
gain_ratio = compute_gain_ratio(data.drop('RESPONSE',axis=1),data['RESPONSE'],info_gain)
gain_ratio_sorted = sorted(gain_ratio.items(), key=lambda x: x[1], reverse=True)
selected_features_gain_ratio = [i[0] for i in gain_ratio_sorted[:12]]
print("Features selected using gain_ratio:", selected_features_gain_ratio)

selected_features_indices_gain_ratio = [data.columns.get_loc(c) for c in selected_features_gain_ratio if c in data]
selected_features_indices_gain_ratio.sort()
selected_features_indices_gain_ratio = [i+2 for i in selected_features_indices_gain_ratio]
print("Indices of features selected using gain_ratio:", selected_features_indices_gain_ratio)

Features selected using gain_ratio: ['AMOUNT', 'CHK_ACCT', 'FOREIGN', 'HISTORY', 'PROP_UNKN_NONE', 'DURATION', 'SAV_ACCT', 'USED_CAR', 'OWN_RES', 'OTHER_INSTALL', 'REAL_ESTATE', 'EDUCATION']
Indices of features selected using gain_ratio: [2, 3, 4, 6, 9, 11, 12, 21, 22, 24, 26, 31]


In [145]:
cor = data.corr()
cor_target = abs(cor["RESPONSE"])

selected_features_cor = cor_target.sort_values(ascending=False)[:13].index.values
selected_features_cor = np.delete(selected_features_cor,0).tolist()
print("Features selected using Correlation:",selected_features_cor)


selected_features_indices_cor = [data.columns.get_loc(c) for c in selected_features_cor if c in data]
selected_features_indices_cor.sort()
selected_features_indices_cor = [i+2 for i in selected_features_indices_cor]
print("Indices of features selected using Correlation:", selected_features_indices_cor)

Features selected using Correlation: ['CHK_ACCT', 'HISTORY', 'DURATION', 'SAV_ACCT', 'AMOUNT', 'OWN_RES', 'PROP_UNKN_NONE', 'REAL_ESTATE', 'EMPLOYMENT', 'OTHER_INSTALL', 'RADIO/TV', 'USED_CAR']
Indices of features selected using Correlation: [2, 3, 4, 6, 8, 11, 12, 13, 21, 22, 24, 26]


In [150]:
selected_features = list(set().union(selected_features_gini , selected_features_info_gain , selected_features_gain_ratio , selected_features_cor ))
print("Features selected:",selected_features)


selected_features_indices = [data.columns.get_loc(c) for c in selected_features if c in data]
selected_features_indices.sort()
selected_features_indices = [i+2 for i in selected_features_indices]
print("Indices of features selected:", selected_features_indices)

Features selected: ['FOREIGN', 'DURATION', 'REAL_ESTATE', 'CHK_ACCT', 'PROP_UNKN_NONE', 'RADIO/TV', 'EDUCATION', 'AMOUNT', 'OWN_RES', 'AGE', 'EMPLOYMENT', 'USED_CAR', 'OTHER_INSTALL', 'HISTORY', 'SAV_ACCT']
Indices of features selected: [2, 3, 4, 6, 8, 9, 11, 12, 13, 21, 22, 23, 24, 26, 31]
